In [1]:
#colab 刪除資料夾
import shutil

! ls
shutil.rmtree('sample_data/')
! ls

sample_data


In [2]:
# https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_object_detection.ipynb#scrollTo=2JCeQU3fkayh
# This Colab requires TF 2.5.

!pip install -U "tensorflow>=2.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.9 kB/s 
     |████████████████████████████████| 1.6 MB 48.1 MB/s 
     |████████████████████████████████| 438 kB 67.0 MB/s 
     |████████████████████████████████| 5.8 MB 41.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [3]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [4]:
# @title Run this!!

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

IMAGES_FOR_TEST = {
  'Beach' : 'models/research/object_detection/test_images/image2.jpg',
  'Dogs' : 'models/research/object_detection/test_images/image1.jpg',
  # By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
  'Naxos Taverna' : 'https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  'Beatles' : 'https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg',
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  'Phones' : 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  'Birds' : 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg',
}

COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

In [4]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3398, done.
remote: Counting objects: 100% (3398/3398), done.
remote: Compressing objects: 100% (2821/2821), done.
remote: Total 3398 (delta 896), reused 1411 (delta 520), pack-reused 0
Receiving objects: 100% (3398/3398), 34.94 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (896/896), done.


In [5]:
%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1694695 sha256=7c78be441c5e20948bec01d0abc8f7ac8c84ff6d76330a6d76cee4e2ae78b1e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-0cxdu4wn/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=2e94d4ca5f2c70de02f8cfb7ccb895421b3073bb8c94afb2ee48d6758090d5ec
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/0



  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.13 requires dill>=0.3.5.1, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.

#  <span style="color:blue"> Tensorflow Object Detection API</span>

https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/

https://github.com/tensorflow/models


#### slim
https://github.com/google-research/tf-slim

### 安裝額外package
<pre>
cd research

pip install .

conda -c conda-forge pycocotools
</pre>

## <font color="purple">下載與安裝Tensorflow Object Detection API於Windows 10與Anaconda</font>
在官方<a href="https://github.com/tensorflow/models" target="_blank" title ="Tensorflow object detection">文件</a>或<a href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb" target="_blank" title="object_detection_tutorial.jpynb">object_detection_tutorial.jpynb</a>有說明安裝在MacOS與Linux作業系統的步驟(比較單純)。這裡著重安裝在Windows 10與Anaconda上的步驟。

### 步驟
1. 從https://github.com/tensorflow/models 下載models-master.zip並解壓縮至你要的路徑(假設為models)
2. 從https://github.com/protocolbuffers/protobuf/releases 下載protoc-3.11.3-win64.zip。
3. 將protoc-3.11.3-win64.zip裡bin\protoc.exe解壓到models\research，因此models\research\bin內有protoc.exe這個檔。
4. 對每一個在models\research\object_detection\protos下的.proto檔轉出相對應的.py檔。可以在models\research下用文字編輯器建立一個內容如下的cvt.bat檔，然後再執行cvt.bat。
<pre>
bin\protoc object_detection\protos\anchor_generator.proto --python_out=.
bin\protoc object_detection\protos\argmax_matcher.proto --python_out=.
bin\protoc object_detection\protos\bipartite_matcher.proto --python_out=.
bin\protoc object_detection\protos\box_coder.proto --python_out=.
bin\protoc object_detection\protos\box_predictor.proto --python_out=.
bin\protoc object_detection\protos\calibration.proto --python_out=.
bin\protoc object_detection\protos\eval.proto --python_out=.
bin\protoc object_detection\protos\faster_rcnn.proto --python_out=.
bin\protoc object_detection\protos\faster_rcnn_box_coder.proto --python_out=.
bin\protoc object_detection\protos\flexible_grid_anchor_generator.proto --python_out=.
bin\protoc object_detection\protos\graph_rewriter.proto --python_out=.
bin\protoc object_detection\protos\grid_anchor_generator.proto --python_out=.
bin\protoc object_detection\protos\hyperparams.proto --python_out=.
bin\protoc object_detection\protos\image_resizer.proto --python_out=.
bin\protoc object_detection\protos\input_reader.proto --python_out=.
bin\protoc object_detection\protos\keypoint_box_coder.proto --python_out=.
bin\protoc object_detection\protos\losses.proto --python_out=.
bin\protoc object_detection\protos\matcher.proto --python_out=.
bin\protoc object_detection\protos\mean_stddev_box_coder.proto --python_out=.
bin\protoc object_detection\protos\model.proto --python_out=.
bin\protoc object_detection\protos\multiscale_anchor_generator.proto --python_out=.
bin\protoc object_detection\protos\optimizer.proto --python_out=.
bin\protoc object_detection\protos\pipeline.proto --python_out=.
bin\protoc object_detection\protos\post_processing.proto --python_out=.
bin\protoc object_detection\protos\preprocessor.proto --python_out=.
bin\protoc object_detection\protos\region_similarity_calculator.proto --python_out=.
bin\protoc object_detection\protos\square_box_coder.proto --python_out=.
bin\protoc object_detection\protos\ssd.proto --python_out=.
bin\protoc object_detection\protos\ssd_anchor_generator.proto --python_out=.
bin\protoc object_detection\protos\string_int_label_map.proto --python_out=.
bin\protoc object_detection\protos\target_assigner.proto --python_out=.
bin\protoc object_detection\protos\train.proto --python_out=.
</pre>
5. 開命令列視窗在models\research目錄下執行指令 (注意虛擬環境設定)

       pip install .
       
6. 若安裝成功，應該可以正確執行下面例子。

In [6]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(model_name):
    base_url   = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir  = tf.keras.utils.get_file(fname=model_name, origin=base_url + model_file, untar=True)
    model_dir  = os.path.join(model_dir,"saved_model")
    model      = tf.saved_model.load(model_dir)
    model      = model.signatures['serving_default']
    return model

def run_inference_for_single_image(model, image):
    
    image = np.asarray(image)
    
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
    output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
    if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    print(image_np.shape)
  # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)=
  # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

    display(Image.fromarray(image_np))

In [7]:
# load label names
PATH_TO_LABELS = tf.keras.utils.get_file(fname='mscoco_label_map.pbtxt', origin='https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt',cache_subdir='datasets/tensorflow/object_detection/data')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

5056/5056 [==============================] - 0s 0us/step


In [8]:
#load test images from the offical website
test_image_url = ['https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/image1.jpg',
                  'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/test_images/image2.jpg']  
TEST_IMAGE_PATHS=[]
for url in test_image_url:
    TEST_IMAGE_PATHS.append(tf.keras.utils.get_file(fname=url.split('/')[-1], origin=url,cache_subdir='datasets/tensorflow/object_detection/test_images'))  
# you can add the filenames of more test images to TEST_IMAGE_PATHS   

1415684/1415684 [==============================] - 0s 0us/step


In [ ]:
train_x = np.array([np.array(Image.open("vid_4_600.jpg")),np.array(Image.open("vid_4_600.jpg"))])
print(train_x.shape)
train_y = np.array([np.array([286.6397,187.5241,407.9479,232.0286]),np.array([286.6397,187.5241,407.9479,232.0286])])
print(train_y.shape)

FileNotFoundError: ignored

In [ ]:
# conv1 =
# conv1 = tf.keras.layers.BatchNormalization()(conv1)
# tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
my_model = tf.keras.models.Sequential()
my_model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(
    3, 3), padding="same", activation='relu', input_shape=train_x.shape[1:]))  # [380, 676, 3]
my_model.add(tf.keras.layers.BatchNormalization())
my_model.add(tf.keras.layers.Conv2D(
    16, (5, 5), dilation_rate=6, padding='same', activation='relu'))
# my_model.add(tf.keras.layers.Conv2D(
#     64, (3, 3), dilation_rate=18, padding='same', activation='relu'))
# my_model.add(tf.keras.layers.Conv2D(
#     64, (3, 3), padding='same', activation='relu'))
# my_model.add(tf.keras.layers.GlobalAveragePooling1D())
my_model.add(tf.keras.layers.Flatten())
my_model.add(tf.keras.layers.Dense(4, activation='relu'))
# my_model.add(tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'))


my_model.compile(loss='categorical_crossentropy',
                 optimizer='adam', metrics=['accuracy'])
history = my_model.fit(train_x, train_y, batch_size=10, epochs=10)

NameError: ignored

In [ ]:
# 線性堆疊模型
model = Sequential()

# 建立卷積層1
# 輸入數字影像 28x28 的大小，執行一次卷積運算，產生 16 個影像，卷積運算不會改變影像大小，結果還是 28x28
# filters=16             建立 16 個 filter weight
# kernel_size=(5,5)      每一個濾鏡大小為 5x5
# padding='same'         讓卷積運算產生的影像大小不變
# input_shape=(28,28,1)  第1, 2 維，是輸入的影像形狀 28x28，第 3 維，因為是單色灰階影像，所以是 1
# activation='relu'      設定 ReLU 激活函數
model.add(Conv2D(filters=16,
                 kernel_size=(5,5),
                 padding='same',
                 input_shape=(28,28,1),
                 activation='relu'))

# 建立池化層
# 輸入參數 pool_size=(2, 2)，執行第一次縮減取樣，將 16 個 28x28 影像，縮小為 16 個 14x14 的影像
model.add(MaxPooling2D(pool_size=(2, 2)))

# 建立卷積層2
# 執行第二次卷積運算，將原本的 16 個影像，轉換為 36 個影像，卷積運算不會改變影像大小，結果還是 14x14
model.add(Conv2D(filters=36,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))

# 建立池化層2，並加入Dropout 避免 overfitting
# 執行第二次縮減取樣，將 36 個 14x14 的影像，縮小為 36 個 7x7 的影像
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 建立神經網路 (平坦層, 隱藏層, 輸出層)
# 建立平坦層
# 根據池化層2 的結果，共36 個 7x7 影像，轉換為 1維向量，長度是 36x7x7=1764，也就是 1764 個 float，正好對應到 1764 個神經元
model.add(Flatten())
# 建立隱藏層，共有 128 個神經元
model.add(Dense(128, activation='relu'))
# 加入 Dropout(0.5)
# 每次訓練迭代時，會隨機在神經網路中，放棄 50% 的神經元，以避免 overfitting
model.add(Dropout(0.5))
# 建立輸出層
# 共 10 個神經元，對應 0~9 共 10 個數字，並使用 softmax 激活函數進行轉換
# softmax 可將神經元的輸出，轉換為預測每一個數字的機率
model.add(Dense(10,activation='softmax'))


In [ ]:
class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out
    
class ASPPNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv7 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv8 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv9 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')
        self.conv10 = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')

        self.conv11 = tf.keras.layers.Conv2D(48, (1, 1), padding='same', activation='relu')
        self.conv12 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv13 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv14 = tf.keras.layers.Conv2D(1, (1, 1), padding='same', activation=None)

        self.maxpool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

        self.aspp = ASPPBlock()

    def call(self, x):

        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out_enc_mid = out
        out = self.maxpool(out)
        out = self.conv7(out)
        out = self.conv8(out)
        out = self.maxpool(out)
        out = self.conv9(out)
        out = self.conv10(out)

        out = self.aspp(out)

        out = tf.image.resize(out, tf.shape(out_enc_mid)[1:3], tf.image.ResizeMethod.BILINEAR)

        out_enc_mid = self.conv11(out_enc_mid)

        out = tf.concat([out, out_enc_mid], axis=3)

        out = self.conv12(out)
        out = self.conv13(out)
        out = self.conv14(out)

        out = tf.image.resize(out, tf.shape(x)[1:3], tf.image.ResizeMethod.BILINEAR)
        out = tf.nn.sigmoid(out)
        return out
    
model = ASPPNet()
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.build(input_shape=[None, 256, 256, 3])

history = model.fit(train_x, train_y, verbose=2, epochs=10, steps_per_epoch=25)

Epoch 1/10


TypeError: ignored

### Object Detection

In [ ]:

drnn_model2 = tf.keras.models.Sequential()
drnn_model2.add(tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape =[None, 3]) )
drnn_model2.add(tf.keras.layers.SimpleRNN(72, return_sequences=True) )
drnn_model2.add(tf.keras.layers.SimpleRNN(72))
drnn_model2.add(tf.keras.layers.Dense(4))



In [9]:
detection_model = load_model('ssd_mobilenet_v2_coco_2018_03_29')

print(detection_model.inputs)
print(detection_model.output_dtypes)

for image_path in TEST_IMAGE_PATHS:
    show_inference(detection_model, image_path)

Output hidden; open in https://colab.research.google.com to view.

### Instance Segmentation

In [10]:
masking_model = load_model('mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28')
print(masking_model.output_shapes)

for image_path in TEST_IMAGE_PATHS:
    show_inference(masking_model, image_path)

Output hidden; open in https://colab.research.google.com to view.

## <font color="purple">安裝影像標註工具(Annotation Tools)</font>

### 工具:labelImg
可標註目標物範圍矩形區域(bounding box)，適合用來標註物件偵測器的訓練影像，輸出格式可為PASCAL VOC與YOLO格式。
#### labelImg安裝
+ <a href="https://github.com/tzutalin/labelImg" target="_blank" title="labelImg">labelImg下載</a>
+ <a href="https://github.com/tzutalin/labelImg#windows--anaconda" target="_blank" title="labelImg on Windows 10 + Anaconda">在Windows10與Anaconda安裝labelImg</a>
  + <a href="https://anaconda.org/anaconda/pyqt" target="_blank" title="Qt Installation">安裝PyQt</a>。
  + 在命令列視窗裡，labelImg的目錄下(這個目錄裡有resources.qrc這個檔案)執行下面指令，將labelImg的.qrc資源檔轉成.py檔。
    
          pyrcc5 -o libs/resources.py resources.qrc
          
#### labelImg執行          
在命令列視窗裡，labelImg的目錄下執行下面指令，就可以啟動labelImg。
    
          python labelImg.py [IMAGE_PATH] [PRE-DEFINED CLASS FILE]
+ 善用labelImg的<a href="https://github.com/tzutalin/labelImg#hotkeys" target="_blank" title="Hot Keys">快速鍵</a>。  
+ 解說labelImg使用的<a href="https://youtu.be/K_mFnvzyLvc?t=9m13s" target="_blank" title="labelImg">視訊</a>。
<div>
    <img src="attachment:LabelImg.png" width="400" align="left">
</div>    

## <font color="purple">訓練自己的物件偵測器</font>
詳細步驟請參考這份<a href="https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html" target="_blank" title="Training Custom Object Detector">文件</a>。將標註工具(labelImg)、上面安裝的模組(models)、訓練資料集(fish-dataset)安排成下面資料夾結構:
<pre>
TensorFlow
├─ addons
│   └─ labelImg
├─ models
│   ├─ official
│   ├─ research
│   ├─ samples
│   └─ tutorials
├─ scripts
│   └─ preprocessing
└─ workspace
    └─ fish_dataset
        ├─ annotations
        ├─ images
        │   ├─ test
        │   └─ train
        ├─ pre-trained-model
        └─ training
</pre>

步驟摘要說明:

### 步驟1:準備訓練資料
訓練影像若放在fish-dataset資料夾裡，安排其子資料夾如下目錄結構:
<pre>
fish-dataset
├─ annotations
├─ images
│   ├─ test
│   └─ train
├─ pre-trained-model
└─ training
</pre>

  + annotations:將存放有關標註資訊的.csv檔與相對應*.record檔(TFRecord)。

  + images:

    + images\train:所有訓練影像及其相對應的.xml檔(若影像已標示)複製到這個資料夾。
  
    + images\test:所有測試影像及其相對應的.xml檔(若影像已標示)複製到這個資料夾
  
  + pre-trained-model:將擺放pretrain model及checkpoint之用。

  + training:將擺放訓練設定參數的.config設定檔及一個標籤流水號與物件名稱對應檔.pbtxt。
  
### 步驟2:建立標籤流水號與物件名稱對映檔label_map.pbtxt並存放在annotations資料夾。
此例標籤流水號與物件名稱對映檔，放在annotations資料夾檔名為label_map.pbtxt內容如下範例:
<pre>
item {
    id: 1
    name: 'salman'
}
item {
    id: 2
    name: 'swordfish'
}
</pre>

### 步驟3:標記影像
使用labelImg或其他工具標記目標物在影像內的位置。  

### 步驟4:將標註資訊轉換成TFRecord格式

官網準備兩個轉檔程式，將它們存到scripts/preprocessing下，照著做，再改一點程式應該會成功。
+ <a href="https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#converting-xml-to-csv"> xml_to_csv.py</a>
+ <a href="https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#converting-from-csv-to-record"> generate_tfrecord.py</a>

下面有一個簡化與整合的程式，將它存在xml_to_tfrecord.py (放在train、test下打指令比較快)。
在命令列，切換到train目錄下執行它。
<pre>
     python xml_to_tfrecord.py
</pre>
那麼在annotations目錄內會產生train.record、label_map.pdtxt。若切換到test目錄執行
<pre>
     python xml_to_tfrecord.py
</pre>
那麼在annotations目錄內會產生test.record、label_map.pdtxt。

<pre>
"""
Usage:
# Create train data:
python xml_to_tfrecord.py -i [PATH_TO_IMAGES_FOLDER]/train -o [PATH_TO_ANNOTATIONS_FOLDER]/train.record -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map_file.pbtxt
or
# Create train.record and label_map.pbtxt in [PATH_TO_ANNOTATIONS_FOLDER]
python xml_to_tfrecord.py 

# Create test data:
python xml_to_tfrecord.py -i [PATH_TO_IMAGES_FOLDER]/test -o [PATH_TO_ANNOTATIONS_FOLDER]/test.record -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map_file.pbtxt
or
# Create train.record and label_map.pbtxt in [PATH_TO_ANNOTATIONS_FOLDER]
python xml_to_tfrecord.py 
"""
from __future__ import division,print_function,absolute_import

import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET
import io
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text)
                    )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df      = pd.DataFrame(xml_list, columns=column_name)
    return xml_df
 
  
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, label_map_dict, path):
    if tf.__version__>='2.0.0':
      with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    else:
      with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
	
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(label_map_dict[row['class']]) 

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main():
    # Initiate argument parser
    parser = argparse.ArgumentParser(description="Sample TensorFlow XML-to-TFRecord converter")
    parser.add_argument("-i","--inputDir",  help="Path to the folder where the input .xml files are stored",type=str)
    parser.add_argument("-o","--outputFile",help="Name of output TFRecord file (including path)", type=str)
    parser.add_argument("-l","--labelFile", help="Name of label map proto (including path)", type=str)
    args = parser.parse_args()

	
    if(args.inputDir is None):
        args.inputDir = os.getcwd()
        
    assert(os.path.isdir(args.inputDir))

    if '/' in args.inputDir:
        input_path_lst = args.inputDir.split('/')
    else:
        input_path_lst = args.inputDir.split('\\')
	
    if ':' in input_path_lst[0]:
        to_workspace_path = os.path.join(input_path_lst[0],os.path.sep,*input_path_lst[1:-2])
    else:
        to_workspace_path = os.path.join(*input_path_lst[0:-2])	
        
    if(args.outputFile is None):
        args.outputFile = os.path.join(to_workspace_path,"annotations",input_path_lst[-1]+".record")

    if(args.labelFile is None):
        args.labelFile = os.path.join(to_workspace_path,"annotations","label_map.pbtxt")

	
    xml_df = xml_to_csv(args.inputDir)

    print(args.labelFile)
    label_map_dict = {label_name : idx+1 for idx, label_name in enumerate(sorted(xml_df['class'].unique()))}

    label_map_item = "item {{\n    id: {0}\n    name: '{1}'\n}}\n"	

    with open(args.labelFile,'w') as fp:
        for label_name in sorted(label_map_dict.keys()):            
            fp.write(label_map_item.format(label_map_dict[label_name],label_name))
    
    if tf.__version__>='2.0.0':	
      writer  = tf.io.TFRecordWriter(args.outputFile)
    else:
      writer  = tf.python_io.TFRecordWriter(args.outputFile)
	
    grouped = split(xml_df, 'filename')    
    
    for group in grouped:
        tf_example = create_tf_example(group, label_map_dict, args.inputDir)
        writer.write(tf_example.SerializeToString())

    writer.close()

    print('Successfully created the TFRecords: {}'.format(args.outputFile))    

if __name__ == '__main__':
    main()
</pre>

### 步驟5:準備<a href="https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#configuring-a-training-pipeline" title="config" target="_blank">設定檔</a>  
從<a href="https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md" target="_blank" title="Model Zoo">Tensorflow model zoo</a>挑選一個預先訓練好的模型，從那個模型開始訓練自己的物件偵測器。
<pre>
例如挑選ssd_mobilenet_v2_coco_2018_03_29.tar.gz
解壓縮後pre-training-model目錄下的樣子。
<img src="attachment:modelmask_pre-trained-model_folder.png" width="250">
</pre>
接下來修改pipeline.config然後放到training目錄內。

#### 確認下面項目，是否要修改:

+ num_classes: 2 #<font color="red">此例為2</font>
+ train_config
  + batch_size: 6 #<font color="red">自行斟酌</font>
+ fine_tune_checkpoint: "pre-trained-model/model.ckpt"
+ tain_input_reader
  + input_path: "annotations/train.record"
  + label_map_path: "annotations/label_map.pbtxt"
+ eval_input_reader
  + input_path: "annotations/test.record"
  + label_map_path: "annotations/label_map.pbtxt"  
  
#### 開始訓練前檢查清單
1. annotations目錄
  + label_map.pdtxt
  + test.record
  + train.record
2. training目錄
  + pipeline.config
3. pre-trained-model
<img src="attachment:modelmask_pre-trained-model_folder.png" width="250">


In [ ]:
label_map_item = "item {{\n    id: {0}\n    name: '{1}'\n}}\n"
print(label_map_item.format(1,'abc'))